<div style ="border: blue solid 1px;
             font-family: cambria;
             box-shadow: 0px 0px 100px rgba(0, 0, 0, 0.5);
             background-image: url(1.jpg);
             height: 100%;
             width: 100%;">
<center>
<a href="#" >
    <h1> <font color="white">PROJET DE PYTHON 2A - ENSAE IP Paris</font> </h1> 
    <h2> <font color="white">Thème : Determinants des choix d'orientation des etudiants dans les etablissments d'enseignement superieur</font> </h2> 
<a href="#" >
    <img src="ensae_picture.PNG" style="float:right; max-width: 190px; display: inline" alt="ENSAE ; height: 100%;" /></a>     
</center>
</div>

In [3]:
#Appel du notebook contenant les packages
%run ./head.ipynb
add_notebook_menu()

In [ ]:
#Ajout de ligne

## <center>1. Importation des données</center>

In [ ]:
#Racine principale de récupération des données
glob_path = "https://data.enseignementsup-recherche.gouv.fr/api/records/1.0/search/?dataset="

### 1.2 Vœux de poursuite d'études et de réorientation dans l'enseignement supérieur et réponses des établissements



https://data.enseignementsup-recherche.gouv.fr/explore/dataset/fr-esr-parcoursup/information/

###### On a 12760 lignes d'individus et l'Api ne permet pas de récupérer plus de 10 000 lignes et donc on a troué comme astuce de récupérer selon le type de l'établissement avant de merger les bases


In [ ]:
#Segmentation pour pouvoir tout récupérer car l'API n'autorise pas plus de 10 000 lignes 
type_etablissement = ["2_Lyc%C3%A9es", "1_Universit%C3%A9s", "3_Autres+formations"]


In [ ]:
#Le lien de récupération
voeux_poursuite = glob_path+"fr-esr-parcoursup&rows=6020&sort=tri&facet=session&facet=contrat_etab&facet=cod_uai&facet=g_ea_lib_vx&facet=dep_lib&facet=region_etab_aff&facet=acad_mies&facet=select_form&facet=fili&facet=form_lib_voe_acc&facet=regr_forma&facet=fil_lib_voe_acc&facet=detail_forma&facet=tri&facet=cod_aff_form&q="


In [ ]:
#Récupération selon le type d'établissement
data_etablissement = [requests.get(voeux_poursuite + type_etablissement[x]) for x in range(3)]

In [ ]:
#Vérification
data_etablissement

In [ ]:
#Conversion de json à pd.Dataframe
data_etablissement = [pd.json_normalize(x.json()['records'][:]) for x in data_etablissement ]

In [ ]:
glob_data = pd.concat(data_etablissement)
glob_data.columns = [c.replace('fields.', '') for c in list(glob_data.columns) ]

# -------------------------------------------- DOULO---------------------------

### Fouille des données et Statistiques descriptives 

In [ ]:
glob_data.shape[1]

In [ ]:
df = glob_data.copy()


In [ ]:
#A utiliser pour comprendre la signification des variables pas explicites : names['varname'] -->signification
names = {'session' : 'session' ,
        'cod_uai' : 'Identifiant_del_établissement' , 
        'g_ea_lib_vx' : 'Identifiant del_établissement' , 
        'dep' : 'code département al del_établisseme nt' , 
        'dep_lib' : 'département de l_établissement' , 
        'region_etab_aff' : 'Région de l_établissement' , 
        'acad_mies' : 'académie de l_établissement' , 
        'fili':'filière de formation très agrégée',
        'contrat_etab':'statut de l_établisseme nt de la filière de formation(public, privé…)', 
        'lib_for_voe_ins':'filière deformation', 
        'select_form':'select_fromat : prend 1 si la formation est sélective, 0 sinon',
        'lib_comp_voe_ins':'filière de formation détaillée', 
        'detail_forma':'filière de formationtrès détaillée', 
        'lien_form_psup':'lien de la formation sur la plateforme parcoursup', 
        'géolocalisation_ des_formations':'coordonnées gps de la formation', 
        'capa_fin':'capacité de l_établisseme nt parformation', 
        'voe_tot':'Effectif total des candidatspour une formation', 
        'voe_tot_f':'dont effectif des candidatespour une formation', 

  'nb_voe_pp' : 'Effectif total des candidats en phaseprincipale' ,  
  'nb_voe_pp_internat' : 'Effectif total des candidats ayant postulé en internat' ,  
  'nb_voe_pp_bg' : 'Effectif des candidats en terminale générale en phase principale' ,  
  'nb_voe_pp_bg_brs' : 'dont effectif des candidats boursiers en terminale générale en phase principale' ,  
  'nb_voe_pp_bt' : 'Effectif des candidats néo en terminale technologique en phase principale' ,  
  'nb_voe_pp_bt_brs' : 'dont effectif des candidats boursiers en terminale technologiqu e en phaseprincipale' ,  
  'nb_voe_pp_bp' : 'Effectif des candidats néo bacheliers en terminalepro en phase principale' ,  
  'nb_voe_pp_bp_brs' : 'dont effectif des candidats boursiers néo bacheliers en terminale pro en phaseprincipale' ,  
  'nb_voe_pp_at' : 'Effectif des autres candidats en phaseprincipale' ,  
  'nb_voe_pc' : 'Effectif total des candidats en phasecomplément aire' ,  
'nb_voe_pc_bg' : 'Effectif des candidats en terminale générale en' ,
  '' : 'phase complémentaire' ,  
  'nb_voe_pc_bt' : 'Effectif des candidats en terminale technologiqu e en phase complémentaire' ,  
  'nb_voe_pc_bp' : 'Effectif des candidats en terminale pro en phase complémentaire' ,  
  'nb_voe_pc_at' : 'Effectifs des autres candidats en phase complémentaire' ,  
  'nb_cla_pp' : 'Effectif total des candidats classés par l_établisseme nt en phaseprincipale' ,  
  'nb_cla_pc' : 'Effectif des candidats classés par l_établisseme nt en phase complémentaire' ,  
  'nb_cla_pp_internat' : 'Effectif des candidats classés par l_établisseme nt en internatcpgE' ,  
  'nb_cla_pp_pasinternat' : 'Effectif des candidats classés par l_établisseme nt hors internat(cpgE)' ,  
  'nb_cla_pp_bg' : 'Effectif des candidats en terminale générale classés parl_établisseme nt' ,  
  'nb_cla_pp_bg_brs' : 'dont effectif des candidats boursiers en terminale générale classés par l_établissement' ,  
  'nb_cla_pp_bt' : 'Effectif des candidats en terminale technologiqu e classés par l_établissement' ,  
  'nb_cla_pp_bt_brs' : 'dont effectif des candidats boursiers en terminale technologiqu e classés par l_établissement' ,  
  'nb_cla_pp_bp' : 'Effectif des candidats en terminale pro classés par l_établissement' ,  
  'nb_cla_pp_bp_brs' : 'dont effectif des candidats boursiers en terminale pro classés parl_établisseme nt' ,  
  'nb_cla_pp_at' : 'Effectif des autres candidats classés parl_établisseme nt' ,  
  'prop_tot' : 'Effectif total des candidats ayant reçu une proposition d_admission de la part del_établisseme nt' ,  
  'prop_tot_bg' : 'Effectif des candidats en terminale générale ayant reçu une proposition d_admission de la part de l_établissement' ,  
  'prop_tot_bg_brs' : 'dont effectif des candidats boursiers en terminale générale ayant reçu une proposition d_admission de la part de l_établissement' ,  
  'prop_tot_bt' : 'Effectif des candidats en terminale technologiqu e ayant reçu une propositiond_admission' ,  
  '' : 'de la part de l_établissement' ,  
  'prop_tot_bt_brs' : 'dont effectif des candidats boursiers en terminale technologiqu e ayant reçu une proposition d_admission de la part del_établisseme nt' ,  
  'prop_tot_bp' : 'Effectif des candidats en terminale professionne lle ayant reçu une proposition d_admission de la part del_établisseme nt' ,  
  'prop_tot_bp_brs' : 'dont effectif des candidats boursiers en terminale générale professionnelle ayant reçu une proposition d_admission de la part de l_établissement' ,  
  'prop_tot_at' : 'Effectif des autres candidats ayant reçu une proposition d_admission de la part de l_établissement' ,  
  'acc_tot' : 'Effectif total des candidats ayant accepté la proposition del_établisseme nt (admis)' ,  
  'acc_tot_f' : 'dont effectif des candidates admises' ,  
  'acc_pp' : 'Effectif des admis en phase principale' ,  
  'acc_pc' : 'Effectif des admis en phase complémentaire' ,  
  'acc_internat' : 'dont effectif des admis en internat' ,  
  'acc_brs' : 'dont effectif des admis boursiers néo bacheliers' ,  
  'acc_debutpp' : 'dont effectif des admis ayant reçu leur proposition d_admission le 15 mai (ouverture pp)' ,  
  'acc_datebac' : 'dont effectif des admis ayant reçu leur proposition d_admission avant le baccalauréat (18 juin)' ,  
  'acc_finpp' : 'dont effectif des admis ayant reçu leur proposition d_admission avant la fin de la procédure principale (19 juillet)' ,  
  'acc_neobac' : 'Effectif des admis néobacheliers' ,  
  'acc_bg' : 'Effectif des admis néo bacheliers généraux' ,  
  'acc_bt' : 'Effectif des admis néo bacheliers technologiques' ,  
  'acc_bp' : 'Effectif des admis néo bacheliers professionnels' ,  
  'acc_at' : 'Effectif des autres candidats admis' ,  
  'acc_mention_non renseignee' : 'dont effectif des admis néo bacheliers sans information sur la mention au bac' ,  
  'acc_sansmention' : 'dont effectif des admis néo bacheliers sans mention au bac' ,  
  'acc_ab' : 'dont effectif des admis néo bacheliers avec mention assez Bien au bac' ,  
  'acc_b' : 'dont effectif des admis néo bacheliers avec mention Bien au bac' ,  
  'acc_tb' : 'dont effectif des admis néo bacheliers avec mention Très Bien au bac' ,  
  'acc_bg_mention' : 'Effectif des admis néo bacheliers généraux ayant eu une mention au bac' ,  
  'acc_bt_mention' : 'Effectif des admis néo bacheliers technologiqu es ayant eu une mention au bac' ,  
  'acc_bp_mention' : 'Effectif des admis néo bacheliers professionne ls ayant eu une mention au bac' ,  
  'acc_term' : 'dont effectif des admis néo bacheliers issus du même établissemen t (BTs/cpgE)' ,  
  'acc_term_f' : 'dont effectif des admises néo bachelières issues du même établissemen t (BTs/cpgE)' ,  
  'acc_aca_orig' : 'dont effectif des admis néo bacheliers issus de la même académie' ,  
  'acc_aca_orig_idf' : 'dont effectif des admis néo bacheliers issus de la même académie (paris/crétei l/versailles réunies)' ,  
  'pct_f' : '% ayant accepté selon le sexe.' ,
  'pct_aca_orig' : '% d_admis néo bacheliers issus de la même académie' ,  
  'pct_aca_orig_idf' : '% d_admis néo bacheliers issus de la même académie (paris/créteil/versailles réunies)' ,  
  'pct_etab_orig' : '% d_admis néo bacheliers issus du même établissemen t (BTs/cpgE)' ,  
  'pct_bours' : '% d_admis néo bacheliers boursiers' ,  
  'pct_acc_debutpp' : '% d_admis ayant reçu leur proposition d_admission le 15 mai (ouverturepp)' ,  
  'pct_acc_datebac' : '% d_admis ayant reçu leur proposition d_admission avant le baccalauréat (18 juin)' ,  
  'pct_acc_finpp' : '% d_admis ayant reçu leur proposition d_admission avant la fin de la procédure principale (19 juillet)' ,  
  'pct_neobac' : '% d_admis néobacheliers' ,  
  'pct_mention_nonrenseignee' : '% d_admis néo bacheliers sans information sur la mention au bac' ,  
  'pct_sansmention' : '% d_admis néo bacheliers sans mention au bac' ,  
  'pct_ab' : '% d_admis néo bacheliers avec mention assez Bien au bac' ,  
  'pct_b' : '% d_admis néo bacheliers avec mention Bien au bac' ,  
  'pct_tb' : '% d_admis néo bacheliers avec mention Très Bien au bac' ,  
  'pct_bg' : '% d_admis néo bacheliers généraux' ,  
  'pct_bg_mention' : 'dont % d_admis avec mention' ,  
  'pct_bt' : '% d_admis néo bacheliers technologiques' ,  
  'pct_bt_mention' : 'dont % d_admis avec mention' ,
  'pct_bp' : '% d_admis néo bacheliers professionnels' ,  
  'pct_bp_mention' : 'dont % d_admis avec mention' ,  
  'lib_grp1' : 'Regroupeme nt 1 effectué par les formations pour les classements' ,  
  'ran_grp1' : 'Rang du dernier appelé du groupe 1' ,  

  'lib_grp2' : 'Regroupement 2 effectué par les formations pour les classements' ,  
  'ran_grp2' : 'Rang du dernier appelé du groupe 2' ,  
  'lib_grp3' : 'Regroupement 3 effectué par les formations pour les classements' ,  
  'ran_grp3' : 'Rang du dernier appelé du groupe 3' ,  
  'lib_grp4' : 'Regroupement 4 effectué par les formations pour les classements' ,  
  'ran_grp4' : 'Rang du dernier appelé du groupe 4' ,  
  'lib_grp5' : 'Regroupement 5 effectué par les formations pour lesc classements' ,  
  'ran_grp5' : 'Rang du dernier appelé du groupe 5' ,  
'taux_adm_psup' : "Taux d’accès des candidats ayant postulé à la formation (ratio entre le dernier appelé et le nombre vœux PP)", 
'taux_adm_psup_pro' : "Dont taux d’accès des candidats ayant un bac PRO ayant postulé à la formation", 
'taux_adm_psup_gen' : "Dont taux d’accès des candidats ayant un bac général ayant postulé à la formation", 
'taux_adm_psup_techno' : "Dont taux d’accès des candidats ayant un bac TECHNO ayant postulé à la formation"
         
  
}





In [ ]:
#df = df.rename(columns = names)
df

In [ ]:
##Modifier les indexes
df.set_index('cod_uai',inplace=True)


In [ ]:
df.dtypes

In [ ]:
# Stockage des types numériques dans une liste
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [ ]:
# Résumé des variables quentitatives
df.select_dtypes(include=numerics).describe()

In [ ]:
#Les doublons : par précaution
df_without_list = df.drop(['g_olocalisation_des_formations', 'geometry.coordinates'], axis = 1)

In [ ]:
df_without_list.duplicated().any()

In [ ]:
#Valeurs manquantes ?
def miss(df) :
    df.isna().sum().sort_values(ascending=False)
    missings = pd.DataFrame(df.isna().sum().sort_values(ascending=False))
    missings.rename(columns = {0 : 'Effectif'}, inplace = True)
    missings['Vars'] = missings.index
    missings.reset_index(drop=True, inplace=True)
    
    missings['Effectif']=missings['Effectif']/glob_data.shape[0]*100
    missings.rename(columns = {'Effectif' : 'Pourcentage de valeurs manquantes'}, inplace = True)
    return missings

In [ ]:
missings = miss(df)

In [ ]:
#Visualisation 
def viz_na(missings, seuil) :
    plt.subplots(figsize=(10, 12))
    sns.set_color_codes("pastel")
    sns.barplot(x="Pourcentage de valeurs manquantes", y='Vars', data=missings[missings['Pourcentage de valeurs manquantes']>seuil])
    plt.title('Pourcentage (%) des valeurs manquantes pour les variables de la base')


In [ ]:
viz_na(missings, seuil = 30)

In [ ]:
#Suppression des variables "non pertinentes"
    #Critère 1 : sum(NA)>30%
var_del1 = list(missings[missings['Pourcentage de valeurs manquantes']>30]['Vars'])
    #Critère 2 : "pas itules pour l'analyse actuellement"
var_del2 = ['geometry.coordinates', 'geometry.type', 'datasetid','recordid','record_timestamp',  'g_olocalisation_des_formations', 'lien_form_psup', 'detail_forma', 'session']
var_del = list(np.unique(var_del1 + var_del2))

In [ ]:
df = df.drop(var_del, axis = 1)

In [ ]:
df.head()

In [ ]:
#Récapitulatif
missings_new = miss(df)


In [ ]:
#Variables avec au moins une valeurs manquantes :  max = 8,5..%
viz_na(missings_new, seuil = 0)

In [ ]:
#Capacité de l'établissement par formation
df['capa_fin'].isna().sum()

#Un établissement

In [ ]:
df[df['capa_fin'].isna()]

In [ ]:
df.drop('0120024L', inplace= True, axis = 'index')

#### Imputation des valeurs manquantes

In [ ]:
#Concentrons nous d'abord sur ces variables

In [ ]:
any_miss = list(missings_new[missings_new['Pourcentage de valeurs manquantes'] > 0]['Vars'])

In [ ]:
for key in any_miss:
    print("{}: {}\n".format(key,names[key]))

In [ ]:
#Ces variables sont liées et peuvent être recalculées avec les données donc pas besoin d'imputation
# taux_adm_psup = sum(rank) / nb_voe_pp
taux = ['taux_adm_psup', 'taux_adm_psup_techno' , 'taux_adm_psup_pro', 'taux_adm_psup_gen']

In [ ]:
df_any_miss = df[any_miss]

In [ ]:
#Relation entre les variables : 

#pct_neobac = pct_bg + pct_bt + pct_bp + Autre
#acc_tot = acc_neobac + Autre
#pct_neobac = pct_bours + Autre = pct_sansmention + pct_ab + pct_b + pct_tb + pact_mention_nonrenseigne

In [ ]:
#On a aucune informations sur les répartitions des admis
cond = (df_any_miss['pct_neobac'].isna()) & ((df_any_miss['pct_bg'].isna()) & (df_any_miss['pct_bt'].isna()) & (df_any_miss['pct_bp'].isna()) )
df[cond]
#Existerait-il des établissements sans admis? 
#Des établissement qui n'ont pas reçu aucune réponse sur leurs propositions
df[df['acc_tot'] ==0]
#S'agit-il de ces établissements : effectivement!
df[df['acc_tot'] ==0].compare(df[cond])
#Il est intéressant de voir ces établissements : ceux qui les caractérisent

In [ ]:
#En excluant ces établissements on peut corriger les valeurs manquantes sur les pct en utilisant les effectif
for key in ['acc_tot','acc_brs', 'acc_neobac', 'acc_ab', 'acc_b', 'acc_tb', 'acc_bg','acc_bg_mention','acc_bt', 'acc_bt_mention', 'acc_bp', 'acc_bp_mention', 'acc_at'] : 
    print("{}: {}\n".format(key,names[key]))         
            
        

In [ ]:
#Pour les établissements ayant au moins un admis, on a pas de valeurs manquantes sur les effectifs désagrégés (bg, bt, bp etc)
#On peut donc corriger les pourcentages!!

viz_na(miss(df[(df['acc_tot']!=0)]), 0)

In [ ]:
df_any_acc = df[df['acc_tot']!=0]
df_any_acc.shape


In [ ]:
#Correction pour les agrégation selon le type de formation
dic = {'pct_bg' : 'acc_bg', 'pct_bt' : 'acc_bt', 'pct_bp' : 'acc_bp'}

In [ ]:
df_any_acc.reset_index(inplace=True)
for pct, eff in dic.items() :
    #Dans le cas où le dénominateur est null, on remplace par 0
    df_any_acc0 = df_any_acc[(df_any_acc[pct].isna()) & (df_any_acc[eff] == 0)].fillna({pct : 0})
    df_any_acc = df_any_acc0.append(df_any_acc[~((df_any_acc[pct].isna()) & (df_any_acc[eff] == 0))].fillna({pct : 0}))
    #Dans le cas où le dénominateur n'est pas null, on applique la formule
    #df_any_acc_not_null = df_any_acc[(df_any_acc[pct].isna()) & (df_any_acc[eff] != 0)].fillna({pct : 100*df_any_acc[eff]/df_any_acc['acc_tot']})
    #Base finale
    #df_any_acc = df_any_acc_null.append(df_any_acc_not_null, ignore_index=True)

In [ ]:
df_any_acc.shape

In [ ]:
#Correction pour les agrégation selon la mention
dic = {'pct_bg_mention' : ['acc_bg_mention', 'acc_bg'], 'pct_bt_mention' : ['acc_bt_mention', 'acc_bt'], 'pct_bp_mention' : ['acc_bp_mention', 'acc_bp']}

In [ ]:

for pct, eff in dic.items() :
    #df_any_acc[pct] = df_any_acc[eff]/df_any_acc['acc_tot']
    
    #df_any_acc[df_any_acc[eff[1]]!=0][pct].fillna(df_any_acc[df_any_acc[eff[1]!=0].isna()][eff[0]]/df_any_acc[~df_any_acc[eff[1]]!=0][eff[1]], inplace=True)
    #df_any_acc[df_any_acc[eff[1]]==0][pct].fillna(0, inplace=True)
    
    #Dans le cas où le dénominateur est null, on remplace par 0
    df_any_acc0 = df_any_acc[(df_any_acc[pct].isna()) & (df_any_acc[eff[1]] == 0)].fillna({pct : 0})
    df_any_acc = df_any_acc0.append(df_any_acc[~((df_any_acc[pct].isna()) & (df_any_acc[eff[1]] == 0))].fillna({pct : 0}))
    #Dans le cas où le dénominateur n'est pas null, on applique la formule
    #df_any_acc = df_any_acc[(df_any_acc[pct].isna()) & (df_any_acc[eff[1]] != 0)].fillna({pct : 100*df_any_acc[eff[0]]/df_any_acc[eff[1]]})


In [ ]:
df_any_acc.shape

In [ ]:
#Il semble que les valeurs manquantes sont seulement pour les établiblissements où pct_neobac=0 (normal), vérifions !
cond = (df_any_acc.pct_mention_nonrenseignee.isna()) & (df_any_acc.pct_bours.isna()) & (df_any_acc.pct_tb.isna()) & (df_any_acc.pct_b.isna()) & (df_any_acc.pct_ab.isna()) & (df_any_acc.pct_sansmention.isna())  & (df_any_acc.pct_b.isna()) & (df_any_acc.pct_aca_orig.isna()) & (df_any_acc.pct_aca_orig_idf.isna())

In [ ]:
var = ['pct_tb', 'pct_b', 'pct_ab', 'pct_sansmention', 'pct_bours','pct_mention_nonrenseignee',  'pct_aca_orig', 'pct_aca_orig_idf']


In [ ]:
df_any_acc[var][cond]

In [ ]:
df_any_acc[var][df_any_acc['pct_neobac']==0]

In [ ]:
#Les bases sont identiques
df_any_acc[['pct_neobac', 'pct_tb', 'pct_b', 'pct_ab', 'pct_sansmention', 'pct_bours', 'pct_mention_nonrenseignee',  'pct_aca_orig', 'pct_aca_orig_idf']][df_any_acc['pct_neobac']==0].compare(df_any_acc[['pct_neobac', 'pct_tb', 'pct_b', 'pct_ab', 'pct_sansmention', 'pct_bours', 'pct_mention_nonrenseignee',  'pct_aca_orig', 'pct_aca_orig_idf']][cond])
#Donc on remplace seulement les NA par 0 

In [ ]:
df_any_acc[var] = df_any_acc[var].apply(lambda x : x.fillna(0))

In [ ]:
df_any_acc[var][df_any_acc['pct_neobac']==0]

In [ ]:
viz_na(miss(df_any_acc), 0)

In [ ]:
#df_any_acc[(df_any_acc.taux_adm_psup_techno.isna()) & (df_any_acc.taux_adm_psup_gen.isna()) & (df_any_acc.taux_adm_psup_pro.isna()) & (df_any_acc.taux_adm_psup.isna())]
df_any_acc[(df_any_acc.taux_adm_psup.isna())]

In [ ]:
df_any_acc.pct_bours.isna().sum()

In [ ]:
#Etablissements où tous les admis sont issus de bac prof
df_any_acc[(df_any_acc['pct_bg'].isna()) & (df_any_acc['pct_bt'].isna()) & (df_any_acc['acc_bp'] ==df_any_acc['acc_neobac'] ) ]

In [ ]:
matrix = 1*df_any_miss.isna()

In [ ]:
matrix[taux]

In [ ]:
#Les autres variables qui ne sont pas les taux
matrix[df_any_miss.columns[~df_any_miss.columns.isin(taux)]]

In [ ]:
#1. Méthodes graphiques : Box-plot, nuage de points, histogrammes...

In [ ]:
#histogramme
def graph(data):
    for colonne in data.columns:
        print('***************************************************************************')
        print('             :------------',colonne,'------------:')
        print('***************************************************************************')
        df[colonne].hist()
        plt.show()

In [ ]:
graph(df_n)

In [ ]:
###Corrélation_Matrice
corrMatrix = df_n.corr()
sns.heatmap(corrMatrix, annot=False)
fig = plt.gcf()
fig.set_size_inches(20, 20)
plt.savefig('Matrice_Correlation.png')

In [ ]:
### STATISTIQUES

In [ ]:
## Effectif total des candidats pour une formation : voe_tot

In [ ]:
#selon le sexe

df['voe_tot_%_filles'] = 100*df['voe_tot_f']/df['voe_tot']

In [ ]:
df['voe_tot_%_filles'].describe()

In [ ]:
df['g_ea_lib_vx'][df['voe_tot_%_filles'] == 0]

In [ ]:
sns.barplot(x="voe_tot",  y="tri", data=df, ci=None)


In [ ]:
sns.barplot(x="voe_tot_f",  y="tri", data=df, ci=None)

In [ ]:
data = df[['g_ea_lib_vx','voe_tot',  'voe_tot_%_filles']].melt('g_ea_lib_vx', var_name='Type', value_name='effectif')

In [ ]:
li = list(df['voe_tot']) + list(df['voe_tot_f'])

In [ ]:
#create DataFrame
df_p = pd.DataFrame({'Valeurs':list(df['voe_tot']) + list(df['voe_tot_f']),
                     
                   'Tri': list(df['tri'])+ list(df['tri'])
                     ,
                     
                   'Iden':['Total']*len(df)+ ['Filles']*len(df)
                               })

In [ ]:
df_p

In [ ]:
sns.barplot(x="Valeurs",  y="Tri", data=df_p, ci=None, hue = 'Iden')

In [ ]:
df["filles"] = df['voe_tot_f']
df["garcons"] = df['voe_tot'] - df['voe_tot_f']
hom = fem = df[['tri','garcons']].groupby('tri').sum()
fem = df[['tri','filles']].groupby('tri').sum()

In [ ]:
base = pd.concat([hom,fem],axis = 1)
base.reset_index(inplace = True)

In [ ]:
base['garcons'] = 100*base['garcons']/(base['garcons'] + base['filles'])

In [ ]:
base['filles'] =100- base['garcons']

In [ ]:
base_ =base.melt('tri', var_name='Sexe', value_name= "Valeurs")
base_

In [ ]:
plt.subplots(figsize=(12, 5))
sx = sns.barplot(x="Valeurs",  y="tri", data=base_, ci=None, hue = 'Sexe')
x_labels = base_['Valeurs']
sx.set 

### 1.3 INSERTION PROFESSIONNEL



https://data.enseignementsup-recherche.gouv.fr/api/records/1.0/search/?dataset=fr-esr-insertion_professionnelle-lp&q=&rows=1500&facet=annee&facet=diplome&facet=numero_de_l_etablissement&facet=etablissement&facet=academie&facet=domaine&facet=code_de_la_discipline&facet=discipline&facet=situation&facet=cle_etab&facet=cle_disc&facet=id_paysage&refine.annee=2018

In [ ]:
insertion = glob_path + "fr-esr-insertion_professionnelle-"
niveau = ["lp", "master"]
complement = "&q=&rows=1500&sort=-annee&facet=annee&facet=diplome&facet=numero_de_l_etablissement&facet=etablissement&facet=academie&facet=domaine&facet=code_de_la_discipline&facet=discipline&facet=situation&facet=cle_etab&facet=cle_disc&facet=id_paysage&refine.annee=2018"

In [ ]:
#Récupération selon le niveau
data_insertion = [requests.get(insertion + niveau[x] + complement) for x in range(2)]

In [ ]:
#Conversion de json à pd.Dataframe
data_insertion = [pd.json_normalize(x.json()['records'][:]) for x in data_insertion ]

In [ ]:
###Les données selon le niveau sont identifiées par la varibale datsetid

In [ ]:
data_insertion[0].shape

In [ ]:
data_insertion[1].shape

In [ ]:
glob_data_insertion = pd.concat(data_insertion)
glob_data_insertion.columns = [c.replace('fields.', '') for c in list(glob_data_insertion.columns) ]

In [ ]:
glob_data_insertion.head()

# ---------------------------------------------KHARIRATOU---------------------------------------------

In [ ]:
df = glob_data_insertion.copy()

In [ ]:
import seaborn as sns

In [ ]:
import numpy as np

In [ ]:
df['na']=np.nan

In [ ]:
df['na']

In [ ]:
df[df['taux_dinsertion']=='ns']

In [ ]:
df['taux_dinsertion'].replace({'ns': np.nan}, inplace = True)

In [ ]:
df[df['taux_dinsertion'] == 'nd'] = np.nan

In [ ]:
df[df['taux_dinsertion'] == 'nd']

In [ ]:
#création variable nombre de personnes insérées
df["taux_dinsertion"] = df["taux_dinsertion"].astype('float64')
df["nombre_de_reponses"] = df["nombre_de_reponses"].astype('float64')
    
df['nb_insertion']= df["taux_dinsertion"]*df["nombre_de_reponses"]

In [ ]:
df_g.reset_index(inplace = True)

In [ ]:
df_g

# Répartition des insérés par discipline et par établissement

In [ ]:
# Répartition du nombre d'insérés par discipline
df_g =df.groupby(['discipline']).sum('nb_insertion')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_insertion", y="discipline", data=df_g)

In [ ]:
df_g =df.groupby(['domaine']).sum('nb_insertion')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_insertion", y="domaine", data=df_g)

In [ ]:
# Répartion du nombre des insérés par établissement
df_g =df.groupby(['cle_etab']).sum('nb_insertion')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_insertion", y="cle_etab", data=df_g)

In [ ]:
# Répartion du nombre des insérés par établissement
df_g =df.groupby(['cle_etab']).sum('nb_insertion')
df_g.reset_index(inplace = True)
df_g

In [ ]:
#import pandas as pd
#pd.crosstab(df.cle_etab, df.nb_insertion).apply(lambda r: r/len(df), axis=1)

#freq = df.groupby(['cle_etab']).sum('nb_insertion').sort_values
#print(freq)

# Caractéristiques des sortants et caractéristiques des sortants insérés

## Par sexe

In [ ]:
#création variable nombre de femmes
df[df['femmes'] == 'ns'] = np.nan
df[df['femmes'] == 'nd'] = np.nan

df["femmes"] = df["femmes"].astype('float64')
df["nombre_de_reponses"] = df["nombre_de_reponses"].astype('float64')
    
df['nb_femmes']= df["femmes"]*df["nombre_de_reponses"]

In [ ]:
# Répartition du nombre de femmes diplomées par discipline
df_g =df.groupby(['discipline']).sum('nb_femmes')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_femmes", y="discipline", data=df_g)

In [ ]:
df_g =df.groupby(['domaine']).sum('nb_femmes')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_femmes", y="domaine", data=df_g)

In [ ]:
# Représentation Proportion des femmes insérées selon l'établissement???
df['nb_femmes_inserees']= df["nb_femmes"]/df["nb_insertion"]
df_g1 =df.groupby(['cle_etab']).sum('nb_femmes_inserees')
#df_g1.reset_index(inplace = True)
df_g1

In [ ]:
# Répartition proportion de femmes insérées par discipline
df_g2 =df.groupby(['discipline']).sum('nb_femmes_inserees')
df_g2.reset_index(inplace = True)
sns.barplot(x="nb_femmes_inserees", y="discipline", data=df_g2)

## Par nombre de boursiers

In [ ]:
#création variable nombre de diplomés boursiers
df[df['de_diplomes_boursiers'] == 'ns'] = np.nan
df[df['de_diplomes_boursiers'] == 'nd'] = np.nan

df["de_diplomes_boursiers"] = df["de_diplomes_boursiers"].astype('float64')
df["nombre_de_reponses"] = df["nombre_de_reponses"].astype('float64')
    
df['nb_de_diplomes_boursiers']= df["de_diplomes_boursiers"]*df["nombre_de_reponses"]

In [ ]:
# nb de diplomés boursiers /etab /discipline
df_g =df.groupby(['discipline']).sum('nb_de_diplomes_boursiers')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_de_diplomes_boursiers", y="discipline", data=df_g)

In [ ]:
df_g =df.groupby(['domaine']).sum('nb_de_diplomes_boursiers')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_de_diplomes_boursiers", y="domaine", data=df_g)

In [ ]:
# Représentation Proportion de boursiers insérés selon l'établissement???
df['nb_boursiers_inseres']= df["nb_de_diplomes_boursiers"]/df["nb_insertion"]
df_g1 =df.groupby(['cle_etab']).sum('nb_boursiers_inseres')
#df_g1.reset_index(inplace = True)
df_g1

In [ ]:
# Répartition proportion de boursiers insérés par discipline
df_g2 =df.groupby(['discipline']).sum('nb_boursiers_inseres')
df_g2.reset_index(inplace = True)
sns.barplot(x="nb_boursiers_inseres", y="discipline", data=df_g2)

## Par nombre d'emplois stables

In [ ]:
#création variable nombre d'emplois stables
df[df['emplois_stables'] == 'ns'] = np.nan
df[df['emplois_stables'] == 'nd'] = np.nan

df["emplois_stables"] = df["emplois_stables"].astype('float64')
df["nombre_de_reponses"] = df["nombre_de_reponses"].astype('float64')
    
df['nb_emplois_stables']= df["emplois_stables"]*df["nombre_de_reponses"]

In [ ]:
# nb de emplois stables /etab /discipline
df_g =df.groupby(['discipline']).sum('nb_emplois_stables')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_emplois_stables", y="discipline", data=df_g)

In [ ]:
df_g =df.groupby(['domaine']).sum('nb_emplois_stables')
df_g.reset_index(inplace = True)
sns.barplot(x="nb_emplois_stables", y="domaine", data=df_g)

In [ ]:
# Représentation Proportion de emplois stables des insérés selon l'établissement???
df['nb_emplois_stables_inseres']= df["nb_emplois_stables"]/df["nb_insertion"]
df_g1 =df.groupby(['cle_etab']).sum('nb_emplois_stables_inseres')
#df_g1.reset_index(inplace = True)
df_g1

In [ ]:
# Répartition proportion de emplois stables des insérés par discipline
df_g2 =df.groupby(['discipline']).sum('nb_emplois_stables_inseres')
df_g2.reset_index(inplace = True)
sns.barplot(x="nb_emplois_stables_inseres", y="discipline", data=df_g2)

## Selon le salaire

In [ ]:
#création variable salaires bruts annuels estimés
df[df['salaire_brut_annuel_estime'] == 'ns'] = np.nan
df[df['salaire_brut_annuel_estime'] == 'nd'] = np.nan

df["salaire_brut_annuel_estime"] = df["salaire_brut_annuel_estime"].astype('float64')

In [ ]:
# Répartition des salaires bruts annuels /discipline
df_g =df.groupby(['discipline']).sum('salaire_brut_annuel_estime')
df_g.reset_index(inplace = True)
sns.barplot(x="salaire_brut_annuel_estime", y="discipline", data=df_g)

In [ ]:
df_g =df.groupby(['domaine']).sum('salaire_brut_annuel_estime')
df_g.reset_index(inplace = True)
sns.barplot(x="salaire_brut_annuel_estime", y="domaine", data=df_g)

In [ ]:
# Représentation de salaires bruts annuels selon l'établissement???
df_g1 =df.groupby(['cle_etab']).sum('salaire_brut_annuel_estime')
#df_g1.reset_index(inplace = True)
df_g1

# BROUILLON

In [ ]:
columns_names = pd.json_normalize(req.json()['facet_groups'][:])

In [ ]:
columns_names = list(ses_2020.columns)
columns_names1 = [c.replace('fields.', '') for c in columns_names ]

In [ ]:
dic=dict()
for i in range(len(columns_names)) :
    dic[columns_names[i]] = columns_names1[i]
ses_2020.rename(dic)